In [37]:
from kaggle_environments import make


In [38]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

In [75]:
game_state = None

def agent(observation, configuration):
    global game_state, game_states

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    
    if observation["step"] == 0:
        game_states = [game_state]
    else:
        game_states.append(game_state)
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)
        actions.append(annotate.circle(0, 0))
    return actions

In [76]:
from kaggle_util import make_env, run, show

env = make_env()
steps = run(env, agent1=agent)

Agent is running!


In [77]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

In [80]:
resource_tiles = find_resources(game_states[22])

cell = resource_tiles[0]
print("Cell at", cell.pos, "has")
print(cell.resource.type, cell.resource.amount)

Cell at (8, 0) has
coal 369


{'seed': 562124210,
 'loglevel': 2,
 'annotations': True,
 'episodeSteps': 361,
 'actTimeout': 3,
 'runTimeout': 1200,
 'mapType': 'random',
 'width': -1,
 'height': -1}